In [ ]:
import folium
import os
import pandas as pd
from typing import List

# --- 使用者設定 ---
TXT_FOLDER = "./txt_data"  # 存放 txt 檔的資料夾
OUTPUT_HTML = "./txt_data/routes_map.html"  # 輸出的地圖 HTML

# --- 地圖初始化 ---
base_map = folium.Map(location=[24.39700, 121.30770], zoom_start=14)

# --- 顏色清單供多條路線使用 ---
colors = ["red", "blue", "green", "purple", "orange", "darkred", "lightblue"]

# --- 讀取每個 TXT 檔 ---
for idx, filename in enumerate(os.listdir(TXT_FOLDER)):
    if filename.endswith(".txt"):
        filepath = os.path.join(TXT_FOLDER, filename)

        # --- 讀取所有行並擷取「路線名稱」 ---
        with open(filepath, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # 搜尋最後一行是否為 # 路線名稱
        route_name = None
        for line in reversed(lines):
            if line.strip().startswith("# 路線名稱："):
                route_name = line.strip().replace("# 路線名稱：", "").strip()
                break

        # 若無路線名稱，則 fallback 使用檔名
        if not route_name:
            route_name = os.path.splitext(filename)[0]

        # --- 讀取資料表 ---
        df = pd.read_csv(filepath, sep="\t", comment="#")  # 忽略註解行

        lat_lon_list = df[["緯度", "經度"]].values.tolist()
        trail_name = df["步道名稱"].iloc[0]

        # --- 新增圖層 ---
        feature_group = folium.FeatureGroup(name=route_name)

        # --- 畫線 ---
        folium.PolyLine(
            lat_lon_list,
            color=colors[idx % len(colors)],
            weight=3,
            opacity=0.8,
            tooltip=route_name,
        ).add_to(feature_group)

        # --- 標記起點與終點 ---
        start_point = lat_lon_list[0]
        end_point = lat_lon_list[-1]

        folium.Marker(
            start_point, popup=f"{route_name} 起點", icon=folium.Icon(color="green")
        ).add_to(feature_group)
        folium.Marker(
            end_point, popup=f"{route_name} 終點", icon=folium.Icon(color="red")
        ).add_to(feature_group)

        # --- 將圖層加入地圖 ---
        feature_group.add_to(base_map)

# --- 加入圖層控制器 ---
folium.LayerControl().add_to(base_map)

# --- 輸出地圖 ---
base_map.save(OUTPUT_HTML)
print(f"地圖已儲存至 {OUTPUT_HTML}")

ValueError: Location values cannot contain NaNs.